In [1]:
import wandb
from datasets import load_dataset, concatenate_datasets
import pandas as pd
from transformers import (
    BertForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
)
from tqdm.auto import tqdm
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, f1_score
from tqdm.auto import tqdm
import torch
from huggingface_hub import notebook_login
import os


In [2]:
full_config = {
    "project": "VKR",
    "config": {
        "dataset": "seara/ru_go_emotions",
        "num_labels": 28,
        "labels": {
            0: "admiration",
            1: "amusement",
            2: "anger",
            3: "annoyance",
            4: "approval",
            5: "caring",
            6: "confusion",
            7: "curiosity",
            8: "desire",
            9: "disappointment",
            10: "disapproval",
            11: "disgust",
            12: "embarrassment",
            13: "excitement",
            14: "fear",
            15: "gratitude",
            16: "grief",
            17: "joy",
            18: "love",
            19: "nervousness",
            20: "optimism",
            21: "pride",
            22: "realization",
            23: "relief",
            24: "remorse",
            25: "sadness",
            26: "surprise",
            27: "neutral",
        },
        "model": "cointegrated/rubert-tiny2",
        "tokenizer": "cointegrated/rubert-tiny2",
        "problem_type": "multi_label_classification",
        "batch_size": 64,
        "epochs": 50,
        "lr": 1e-5,
    },
    "name": "rubert-tiny2-ru-go-emotions",
}
config = full_config["config"]


In [3]:
def binarize_labels(labels, num_labels):
    return [int(i in labels) for i in range(num_labels)]


def label2id(class_labels):
    label2id = {label: i for i, label in enumerate(class_labels)}
    id2label = {i: label for label, i in label2id.items()}
    return label2id, id2label


In [4]:
def calculate_aucs(y_true, y_pred, num_labels):
    return [roc_auc_score(y_true[:, i], y_pred[:, i]) for i in range(num_labels)]


def calculate_f1_score(y_true, y_pred, average, num_labels):
    return [
        f1_score(y_true[:, i], y_pred[:, i] > 0.5, average=average)
        for i in range(num_labels)
    ]


def calculate_metrics(y_true, y_pred, num_labels):
    auc_rocs = calculate_aucs(y_true, y_pred, num_labels)

    f1_scores_micro = calculate_f1_score(y_true, y_pred, "micro", num_labels)

    f1_scores_macro = calculate_f1_score(y_true, y_pred, "macro", num_labels)

    df = pd.DataFrame(
        [auc_rocs, f1_scores_micro, f1_scores_macro],
        columns=config["labels"].values(),
        index=["AUC ROC", "F1 micro", "F1 macro"],
    )
    df["mean"] = df.mean(axis=1)
    # df["mean(emotions)"] = df.drop("no emotion", axis=1).mean(axis=1)
    return df


In [5]:
def predict(model, dataloader):
    with torch.no_grad():
        y_true = []
        y_pred = []
        for batch in dataloader:
            batch = batch.to(model.device)
            output = model(**batch)
            y_true.append(batch.labels)
            y_pred.append(torch.softmax(output.logits, -1))
    return torch.cat(y_true).cpu().numpy(), torch.cat(y_pred).cpu().numpy()


In [6]:
def train(model, train_dataloader, optimizer, epochs, test_dataloader):
    tq = tqdm(range(epochs))

    for epoch in tq:
        model.train()
        for i, batch in enumerate(train_dataloader):
            optimizer.zero_grad()
            batch = batch.to(model.device)
            output = model(**batch)
            loss = output.loss
            loss.backward()
            optimizer.step()

        # if epoch % 5 == 0:
        model.eval()

        y_true, y_pred = predict(model, train_dataloader)
        train_auc = np.mean(calculate_aucs(y_true, y_pred, config["num_labels"]))

        y_true, y_pred = predict(model, test_dataloader)
        test_auc = np.mean(calculate_aucs(y_true, y_pred, config["num_labels"]))

        tq.set_description(f"loss: {loss.item():4.4f}, AUC: {test_auc:4.4f}")
        wandb.log(
            {"train_auc": train_auc, "test_auc": test_auc, "train_loss": loss.item()}
        )


In [7]:
dataset = load_dataset(config["dataset"])


Using custom data configuration seara--ru-go-emotions-010f1c10233a04e9
Found cached dataset parquet (/home/seara/.cache/huggingface/datasets/seara___parquet/seara--ru-go-emotions-010f1c10233a04e9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(config["tokenizer"])


In [9]:
config["label2id"], config["id2label"] = label2id(config["labels"].values())
processed_dataset = dataset.map(
    lambda x: tokenizer(x["text"], truncation=True), batched=True
).map(
    lambda x: {
        "label": [
            float(y) for y in binarize_labels(x["labels"], config["num_labels"])
        ]
    },
    batched=False,
    remove_columns=["text", "labels", "id","ru_text"],
)


Loading cached processed dataset at /home/seara/.cache/huggingface/datasets/seara___parquet/seara--ru-go-emotions-010f1c10233a04e9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-6a398f7cc851dfd3.arrow
Loading cached processed dataset at /home/seara/.cache/huggingface/datasets/seara___parquet/seara--ru-go-emotions-010f1c10233a04e9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-f9ddef66c5d9538a.arrow


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/43410 [00:00<?, ?ex/s]

  0%|          | 0/5426 [00:00<?, ?ex/s]

  0%|          | 0/5427 [00:00<?, ?ex/s]

In [10]:
model = BertForSequenceClassification.from_pretrained(
    config["model"],
    num_labels=config["num_labels"],
    problem_type=config["problem_type"],
    label2id=config["label2id"],
    id2label=config["id2label"],
)


Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

In [11]:
data_collator = DataCollatorWithPadding(tokenizer)

train_dataloader = DataLoader(
    processed_dataset["train"],
    batch_size=config["batch_size"],
    drop_last=False,
    shuffle=True,
    num_workers=0,
    collate_fn=data_collator,
)
test_dataloader = DataLoader(
    processed_dataset["test"],
    batch_size=config["batch_size"],
    drop_last=False,
    shuffle=True,
    num_workers=0,
    collate_fn=data_collator,
)


In [12]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=config["lr"])


In [13]:
os.environ["WANDB_NOTEBOOK_NAME"] = f"{os.getcwd()}/{full_config['name']}.ipynb"
wandb.login()
wandb.init(**full_config)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: seara. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
model.cuda()
train(model, train_dataloader, optimizer, config["epochs"], test_dataloader)


  0%|          | 0/50 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [25]:
calculate_metrics(*predict(model, test_dataloader), config["num_labels"]).round(4)


,no emotion,joy,sadness,surprise,fear,anger,mean,mean(emotions)
AUC ROC,0.9239,0.9550,0.9537,0.9045,0.9016,0.7682,0.9011,0.8973
F1 micro,0.8608,0.9400,0.9336,0.9485,0.9559,0.9192,0.9263,0.9372
F1 macro,0.8527,0.9005,0.8975,0.8402,0.8350,0.6768,0.8338,0.8306


In [ ]:
# notebook_login()


In [26]:
model.push_to_hub(full_config["name"])
tokenizer.push_to_hub(full_config["name"])


CommitInfo(commit_url='https://huggingface.co/seara/rubert-tiny2-cedr/commit/241abd5e1685e91cee1f1843d99cf16c0a8e285f', commit_message='Upload tokenizer', commit_description='', oid='241abd5e1685e91cee1f1843d99cf16c0a8e285f', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
wandb.finish()


test_auc,▁▂▂▄▅▅▆▇▇▇▇▇▇▇██████████████████████████
train_auc,▁▂▂▃▄▄▅▆▆▆▆▇▇▇▇▇▇▇▇█████████████████████
train_loss,█▇▆▆▅▄▅▅▅▄▄▄▃▃▃▂▃▃▃▃▂▂▂▂▂▂▃▂▁▂▂▁▁▂▁▁▁▁▁▁
test_auc,0.90113
train_auc,0.99792
train_loss,0.02599
